In [153]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from collections import defaultdict, Counter
import re
import os
import tensorflow as tf
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Datos de validación

In [154]:
df = pd.read_csv('C:/Users/USER/Documents/Maestria/CAOBA/Laura/data/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv', sep=',', encoding='utf-8') # cargar datos
df.head()

,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text
0,0,ms-val-0,A 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,1,5,Culture tests indicate Neisseria gonorrhoeae.,Culture tests indicate Trichomonas vaginalis.,A 24-year-old woman comes to the emergency dep...
1,1,ms-val-1,A 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,0,-1,NaN,NaN,NaN
2,2,ms-val-2,A 43-year-old woman comes to the physician bec...,0 A 43-year-old woman comes to the physician b...,1,24,Pathology confirms Streptococcus pneumoniae is...,Pathology confirms Mycoplasma pneumoniae is th...,A 43-year-old woman comes to the physician bec...
3,3,ms-val-3,A 4-year-old is brought into the emeregency ro...,0 A 4-year-old is brought into the emeregency ...,1,4,Laboratory studies confirm that he has a respi...,Laboratory studies confirm that he has parainf...,A 4-year-old is brought into the emeregency ro...
4,4,ms-val-4,A previously healthy 25-year-old man comes to ...,0 A previously healthy 25-year-old man comes t...,1,0,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...


In [155]:
# Analizamos las 2 columnas para obtener sus tipos de datos y otras características
column_info = df.dtypes

# Analizamos los nombres de las columnas para identificar las primeras 2
column_names = df.columns[:2]

# Extraemos ejemplos de datos de las primeras 5 columnas para entender mejor su contenido
sample_data = df.head()

column_info[:5], column_names, sample_data

(Unnamed: 0     int64
 Text ID       object
 Text          object
 Sentences     object
 Error Flag     int64
 dtype: object,
 Index(['Unnamed: 0', 'Text ID'], dtype='object'),
    Unnamed: 0   Text ID                                               Text  \
 0           0  ms-val-0  A 24-year-old woman comes to the emergency dep...   
 1           1  ms-val-1  A 24-year-old woman comes to the emergency dep...   
 2           2  ms-val-2  A 43-year-old woman comes to the physician bec...   
 3           3  ms-val-3  A 4-year-old is brought into the emeregency ro...   
 4           4  ms-val-4  A previously healthy 25-year-old man comes to ...   
 
                                            Sentences  Error Flag  \
 0  0 A 24-year-old woman comes to the emergency d...           1   
 1  0 A 24-year-old woman comes to the emergency d...           0   
 2  0 A 43-year-old woman comes to the physician b...           1   
 3  0 A 4-year-old is brought into the emeregency ...           1   
 4

In [156]:
# Lista de columnas a revisar por caracteres especiales
columns_to_check = ['Text', 'Sentences']  

# Definir un patrón de regex para buscar caracteres que no sean alfanuméricos ni espacios básicos
special_char_pattern = re.compile(r'[^a-zA-Z0-9 \.,!?]')

# Función para buscar caracteres especiales 
def find_special_characters(text):
    return special_char_pattern.findall(text)

# Revisar cada columna y fila por caracteres especiales
for column in columns_to_check:
    df[column].apply(lambda x: find_special_characters(x) if pd.notnull(x) else []).apply(lambda x: print(f'Caracteres especiales en {column}: {set(x)}') if x else None)


Caracteres especiales en Text: {'/', "'", '-'}
Caracteres especiales en Text: {'/', "'", '-'}
Caracteres especiales en Text: {'/', ')', '+', '(', '\n', '-', ':'}
Caracteres especiales en Text: {"'", '-'}
Caracteres especiales en Text: {'(', '-', ')'}
Caracteres especiales en Text: {'(', '-', ')'}
Caracteres especiales en Text: {'(', '-', ')'}
Caracteres especiales en Text: {'(', '-', ')'}
Caracteres especiales en Text: {'(', '/', '-', ')'}
Caracteres especiales en Text: {'(', '/', '-', ')'}
Caracteres especiales en Text: {'/', ')', ';', '(', '-'}
Caracteres especiales en Text: {'/', ')', ';', '(', '-'}
Caracteres especiales en Text: {'/', '‰', 'â', ')', ';', '+', '¥', '(', "'", '-'}
Caracteres especiales en Text: {'/', '‰', 'â', ')', ';', '+', '¥', '(', "'", '-'}
Caracteres especiales en Text: {'/', ')', ';', '(', '-'}
Caracteres especiales en Text: {'/', ')', ';', '(', '-'}
Caracteres especiales en Text: {'(', '/', '-', ')'}
Caracteres especiales en Text: {'(', '/', '-', ')'}
Caracter

In [157]:
special_char_pattern = re.compile(r'[^a-zA-Z0-9 \.,!?]')

# Re-definir la lista de columnas a revisar para caracteres especiales
columns_to_check = ['Text', 'Sentences']  

# Contar los caracteres especiales en las columnas de texto seleccionadas
special_characters_counts = df[columns_to_check].applymap(lambda x: len(special_char_pattern.findall(x)) if pd.notnull(x) else 0)

# Calcular el total de caracteres especiales por columna
total_special_characters_per_column = special_characters_counts.sum()

total_special_characters_per_column


Text          6573
Sentences    19822
dtype: int64

In [158]:
# Extraer todos los caracteres especiales encontrados en la columna 'Text'
all_special_characters_text_id = ''.join(df['Text'].apply(lambda x: ''.join(special_char_pattern.findall(x))))

# Contar la frecuencia de cada caracter especial
special_character_frequencies_text_id = Counter(all_special_characters_text_id)

special_character_frequencies_text_id


Counter({'-': 1944,
         "'": 154,
         '/': 1473,
         '(': 451,
         ')': 451,
         ':': 230,
         '\n': 864,
         '+': 117,
         ';': 93,
         'â': 171,
         '‰': 4,
         '¥': 4,
         'Î': 22,
         '¼': 16,
         '=': 18,
         '€': 159,
         '“': 67,
         '%': 163,
         '^': 15,
         '>': 5,
         'œ': 39,
         '\x9d': 39,
         'Ë': 2,
         'š': 2,
         '<': 16,
         '"': 6,
         '™': 14,
         '±': 6,
         'Ã': 6,
         '©': 1,
         'ˆ': 8,
         '’': 8,
         'Ÿ': 5})

In [159]:
# Inspeccionar los caracteres especiales más frecuentes en 'Text' y 'Sentences'
def extract_special_characters(text):
    return special_char_pattern.findall(text)

# Aplicar la extracción a las columnas 'Text' y 'Sentences'
special_characters_text = ''.join(df['Text'].apply(lambda x: ''.join(extract_special_characters(x))))
special_characters_sentences = ''.join(df['Sentences'].apply(lambda x: ''.join(extract_special_characters(x))))

# Contar la frecuencia de cada caracter especial
special_character_frequencies_text = Counter(special_characters_text)
special_character_frequencies_sentences = Counter(special_characters_sentences)

# Identificar los 10 caracteres especiales más comunes en cada columna
top_10_special_characters_text = special_character_frequencies_text.most_common(10)
top_10_special_characters_sentences = special_character_frequencies_sentences.most_common(10)

top_10_special_characters_text, top_10_special_characters_sentences

([('-', 1944),
  ('/', 1473),
  ('\n', 864),
  ('(', 451),
  (')', 451),
  (':', 230),
  ('â', 171),
  ('%', 163),
  ('€', 159),
  ("'", 154)],
 [('\n', 11589),
  ('-', 2822),
  ('/', 2125),
  ('(', 654),
  (')', 654),
  (':', 321),
  ('â', 245),
  ('%', 231),
  ('€', 227),
  ("'", 218)])

In [160]:
text_column = df['Text']

# Definir un patrón de regex para buscar caracteres que no sean alfanuméricos ni espacios básicos
special_char_pattern = re.compile(r'[^a-zA-Z0-9\s]')

# Diccionario para almacenar hasta 5 ejemplos por cada carácter especial encontrado
examples_per_special_char = defaultdict(list)

# Recorrer cada texto en la columna 'Text'
for text in text_column:
    # Encontrar todos los caracteres especiales únicos en el texto actual
    unique_special_chars = set(special_char_pattern.findall(text))
    # Para cada carácter especial encontrado, agregar el texto actual como un ejemplo
    for char in unique_special_chars:
        if len(examples_per_special_char[char]) < 10:  
            examples_per_special_char[char].append(text)

# Convertir el diccionario a un formato más legible para la presentación
{char: examples[:2] for char, examples in examples_per_special_char.items()}

{'/': ["A 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. During this time, she has also had recurrent dizziness and palpitations. The symptoms started while she was at a friend's birthday party, where she had one beer. One week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy. She has no history of major medical illness. Culture tests indicate Neisseria gonorrhoeae. Her pulse is 106/min and blood pressure is 102/73 mm Hg. Physical examination shows facial flushing and profuse sweating. ",
  "A 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. During this time, she has also had recurrent dizziness and palpitations. The symptoms started while she was at a friend's birthday party, where she had one beer. One week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial ther

In [161]:
# Reemplazar caracteres especiales por su equivalente en texto
replacements = {
    "â€\x9d": " ", 
    ".â€\x9d": ".",
    "â€“": "-", 
    "â€œsomething": "something", 
    "patientâ€™s": "patients",
    "â‰¥": "≥", 
    "\n": ",", 
    "â€œ": "", 
    "Waldenstromâ€™s": "Waldenströms", 
    "Âµmol/L": "µmol/L", 
    "â€œmemory": "memory", 
    "weeksâ€™": "weeks", 
    "74 Î¼m3": "74 μm^3", 
    "ÂµIU/L": "µIU/L", 
    "â‰¤": "≤", 
    "Clâˆ’": "Cl-", 
    "physicianâ€™s": "physicians", 
    "ÂºF": "ºF", 
    "âˆ’": "-", 
    "Â½": "½", 
    "Î¼g/dL": "μg/dL", 
    "pediatricianâ€™s": "pediatricians", 
    "yearsâ€™": "years", 
    "Âµg": "µg", 
    "â€": "", 
    "Kennedyâ€™s": "Kennedys", 
    "Kennedy™s": "Kennedys",
    "can™t": "can't",
    "Î²": "β", 
    "Ã—": "x", 
    "ÃŸ-hCG": "β-hCG", 
    "SÃ©quard": "Séquard", 
    "Î³": "γ", 
    "Î¼m3": "μm^3",
    "Î±": "α", 
    "Î¼U/mL": "μU/mL", 
    "Î¼": "μ", 
    "'": " ",
    "\"":"" #Este lo dejo a consideracion, dependiendo si les funciona o no
}

# Aplicar todas las sustituciones
for old, new in replacements.items():
    df['Text'] = df['Text'].str.replace(old, new, regex=False)

# Verificar el reemplazo 
df['Text'].head()


0    A 24-year-old woman comes to the emergency dep...
1    A 24-year-old woman comes to the emergency dep...
2    A 43-year-old woman comes to the physician bec...
3    A 4-year-old is brought into the emeregency ro...
4    A previously healthy 25-year-old man comes to ...
Name: Text, dtype: object

In [ ]:
# Elimina los espacios al inicio y al final del texto en la columna 'Text'
df['Text'] = df['Text'].apply(lambda x: x.strip())

# Busca y reemplaza dos comas seguidas por una sola coma en la columna 'Text'
df['Text'] = df['Text'].str.replace(r',,', ',')

# Busca y reemplaza dos o más espacios seguidos por un solo espacio en la columna 'Text'
df['Text'] = df['Text'].str.replace(r'  +', ' ', regex=True)
df['Text'] = df['Text'].str.replace('Cl- ', 'Cl ', regex=False)

#Remplazo apostrofes
df['Text'] = df['Text'].str.replace("'", "", regex=False)

#Remplazo apostrofes
df['Text'] = df['Text'].str.replace("’", "", regex=False)
df['Text'] = df['Text'].str.replace(':,', ':', regex=False)
df['Text'] = df['Text'].str.replace('mm hg', 'mmHg', flags=re.IGNORECASE, regex=True)
df['Text'] = df['Text'].str.replace(',_', ',', regex=False)
df['Text'] = df['Text'].str.replace('[', '', regex=False)
df['Text'] = df['Text'].str.replace(']', '', regex=False)
df['Text'] = df['Text'].str.replace('mg dl', 'mg/dl', regex=False)
df['Text'] = df['Text'].str.replace('é', 'e', regex=False)

#convertir en minuscula
df['Text'] = df['Text'].str.lower()

In [176]:
text_column = df['Text']

# Definir un patrón de regex para buscar caracteres que no sean alfanuméricos ni espacios básicos
special_char_pattern = re.compile(r'[^a-zA-Z0-9\s]')

# Diccionario para almacenar hasta 5 ejemplos por cada carácter especial encontrado
examples_per_special_char = defaultdict(list)

# Recorrer cada texto en la columna 'Text'
for text in text_column:
    # Encontrar todos los caracteres especiales únicos en el texto actual
    unique_special_chars = set(special_char_pattern.findall(text))
    # Para cada carácter especial encontrado, agregar el texto actual como un ejemplo
    for char in unique_special_chars:
        if len(examples_per_special_char[char]) < 20:  
            examples_per_special_char[char].append(text)

# Convertir el diccionario a un formato más legible para la presentación
{char: examples[:2] for char, examples in examples_per_special_char.items()}

{'/': ['a 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. during this time, she has also had recurrent dizziness and palpitations. the symptoms started while she was at a friend s birthday party, where she had one beer. one week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy. she has no history of major medical illness. culture tests indicate neisseria gonorrhoeae. her pulse is 106/min and blood pressure is 102/73 mmhg. physical examination shows facial flushing and profuse sweating.',
  'a 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. during this time, she has also had recurrent dizziness and palpitations. the symptoms started while she was at a friend s birthday party, where she had one beer. one week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therap

In [177]:
print(df['Text'])

0      a 24-year-old woman comes to the emergency dep...
1      a 24-year-old woman comes to the emergency dep...
2      a 43-year-old woman comes to the physician bec...
3      a 4-year-old is brought into the emeregency ro...
4      a previously healthy 25-year-old man comes to ...
                             ...                        
569    a 20-year-old woman presents to the emergency ...
570    a 20-year-old woman presents to the emergency ...
571    a 26-year-old male comes into your clinic comp...
572    a 56-year-old woman with a history of alcoholi...
573    a 56-year-old woman with a history of alcoholi...
Name: Text, Length: 574, dtype: object


In [178]:
#Esto lo hago con el fin de validar que no haya caracteres especiales, estre codigo va antes y despues de la limpieza de caracteres especiales, con el fin de validar que tenia y que limpio

text_column = df['Text']

# Definir un patrón de regex para buscar caracteres que no sean alfanuméricos ni espacios básicos
special_char_pattern = re.compile(r'[^a-zA-Z0-9\s]')

# Diccionario para almacenar hasta 5 ejemplos por cada carácter especial encontrado
examples_per_special_char = defaultdict(list)

# Recorrer cada texto en la columna 'Text'
for text in text_column:
    # Encontrar todos los caracteres especiales únicos en el texto actual
    unique_special_chars = set(special_char_pattern.findall(text))
    # Para cada carácter especial encontrado, agregar el texto actual como un ejemplo
    for char in unique_special_chars:
        if len(examples_per_special_char[char]) < 30:  
            examples_per_special_char[char].append(text)

# Convertir el diccionario a un formato más legible para la presentación
{char: examples[:2] for char, examples in examples_per_special_char.items()}
examples_per_special_char

defaultdict(list,
            {'/': ['a 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. during this time, she has also had recurrent dizziness and palpitations. the symptoms started while she was at a friend s birthday party, where she had one beer. one week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy. she has no history of major medical illness. culture tests indicate neisseria gonorrhoeae. her pulse is 106/min and blood pressure is 102/73 mmhg. physical examination shows facial flushing and profuse sweating.',
              'a 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. during this time, she has also had recurrent dizziness and palpitations. the symptoms started while she was at a friend s birthday party, where she had one beer. one week ago, the patient was diagnosed with a genitourinary inf

In [179]:
# Inspeccionar los caracteres especiales más frecuentes en 'Text' y 'Sentences'
# Reviso nuevamente los 10 mas comunes, comparandolo con sentences y ver que resultados obtengo
def extract_special_characters(text):
    return special_char_pattern.findall(text)

# Aplicar la extracción a las columnas 'Text' y 'Sentences'
special_characters_text = ''.join(df['Text'].apply(lambda x: ''.join(extract_special_characters(x))))
special_characters_sentences = ''.join(df['Sentences'].apply(lambda x: ''.join(extract_special_characters(x))))

# Contar la frecuencia de cada caracter especial
special_character_frequencies_text = Counter(special_characters_text)
special_character_frequencies_sentences = Counter(special_characters_sentences)

# Identificar los 10 caracteres especiales más comunes en cada columna
top_10_special_characters_text = special_character_frequencies_text.most_common(10)
top_10_special_characters_sentences = special_character_frequencies_sentences.most_common(10)

top_10_special_characters_text, top_10_special_characters_sentences

([('.', 6464),
  (',', 3607),
  ('-', 1999),
  ('/', 1473),
  ('(', 451),
  (')', 451),
  (':', 230),
  ('%', 163),
  ('+', 117),
  (';', 93)],
 [('.', 9335),
  (',', 4117),
  ('-', 2822),
  ('/', 2125),
  ('(', 654),
  (')', 654),
  (':', 321),
  ('â', 245),
  ('%', 231),
  ('€', 227)])

In [180]:
df.head()

,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text
0,0,ms-val-0,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,1,5,Culture tests indicate Neisseria gonorrhoeae.,Culture tests indicate Trichomonas vaginalis.,A 24-year-old woman comes to the emergency dep...
1,1,ms-val-1,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,0,-1,NaN,NaN,NaN
2,2,ms-val-2,a 43-year-old woman comes to the physician bec...,0 A 43-year-old woman comes to the physician b...,1,24,Pathology confirms Streptococcus pneumoniae is...,Pathology confirms Mycoplasma pneumoniae is th...,A 43-year-old woman comes to the physician bec...
3,3,ms-val-3,a 4-year-old is brought into the emeregency ro...,0 A 4-year-old is brought into the emeregency ...,1,4,Laboratory studies confirm that he has a respi...,Laboratory studies confirm that he has parainf...,A 4-year-old is brought into the emeregency ro...
4,4,ms-val-4,a previously healthy 25-year-old man comes to ...,0 A previously healthy 25-year-old man comes t...,1,0,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...


In [181]:
# Estadísticas para 'Text' (resumen categórico)
text_stats = df['Text'].describe()

text_stats


count                                                   574
unique                                                  574
top       a 24-year-old man is brought to the emergency ...
freq                                                      1
Name: Text, dtype: object

In [182]:
# Estadísticas para 'Sentences' (resumen categórico)
sentences_stats = df['Sentences'].describe()

sentences_stats

count                                                   574
unique                                                  574
top       0 A 2-year-old boy presents to the pediatricia...
freq                                                      1
Name: Sentences, dtype: object

## Validación LSTM

In [184]:
model_path = 'C:/Users/USER/Documents/Maestria/CAOBA/Laura/data/my_nlp_model_tf'
loaded_model = tf.keras.models.load_model(model_path)

In [185]:
vocab_path = 'C:/Users/USER/Documents/Maestria/CAOBA/Laura/data/text_vocab.json'

# Cargar el vocabulario
with open(vocab_path) as f:
    vocab = json.load(f)

# Crear una nueva instancia de TextVectorization con el vocabulario cargado
text_vector = tf.keras.layers.TextVectorization(max_tokens=20000, 
                                                output_sequence_length=124)
text_vector.adapt(tf.data.Dataset.from_tensor_slices(["dummy"])) # Inicialización dummy
text_vector.set_vocabulary(vocab)


In [186]:
df.head()

,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text
0,0,ms-val-0,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,1,5,Culture tests indicate Neisseria gonorrhoeae.,Culture tests indicate Trichomonas vaginalis.,A 24-year-old woman comes to the emergency dep...
1,1,ms-val-1,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,0,-1,NaN,NaN,NaN
2,2,ms-val-2,a 43-year-old woman comes to the physician bec...,0 A 43-year-old woman comes to the physician b...,1,24,Pathology confirms Streptococcus pneumoniae is...,Pathology confirms Mycoplasma pneumoniae is th...,A 43-year-old woman comes to the physician bec...
3,3,ms-val-3,a 4-year-old is brought into the emeregency ro...,0 A 4-year-old is brought into the emeregency ...,1,4,Laboratory studies confirm that he has a respi...,Laboratory studies confirm that he has parainf...,A 4-year-old is brought into the emeregency ro...
4,4,ms-val-4,a previously healthy 25-year-old man comes to ...,0 A previously healthy 25-year-old man comes t...,1,0,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...


In [187]:
# Asumiendo que df es tu DataFrame y que ya has cargado 'text_vector' y 'loaded_model'

texts = df['Text'].values
labels = df['Error Flag'].values

# Preprocesar los textos
X_new = text_vector(texts)# Transforma los textos a vectores

In [188]:
# Asumiendo que 'texts' es tu array unidimensional de textos brutos (strings).
new_predictions = loaded_model.predict(texts)

18/18 [==============================] - 2s 35ms/step


In [189]:
predicted_labels = np.round(new_predictions).astype(int)

# Asegurarte de que las etiquetas verdaderas también sean enteros si no lo son ya
true_labels = labels.astype(int)

# Ahora calcular las métricas
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy on new data: {accuracy}")
print(f"Precision on new data: {precision}")
print(f"Recall on new data: {recall}")
print(f"F1 Score on new data: {f1}")


Accuracy on new data: 0.5365853658536586
Precision on new data: 0.5462805123100737
Recall on new data: 0.5365853658536586
F1 Score on new data: 0.5376664132961724


In [190]:
df.head()

,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text
0,0,ms-val-0,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,1,5,Culture tests indicate Neisseria gonorrhoeae.,Culture tests indicate Trichomonas vaginalis.,A 24-year-old woman comes to the emergency dep...
1,1,ms-val-1,a 24-year-old woman comes to the emergency dep...,0 A 24-year-old woman comes to the emergency d...,0,-1,NaN,NaN,NaN
2,2,ms-val-2,a 43-year-old woman comes to the physician bec...,0 A 43-year-old woman comes to the physician b...,1,24,Pathology confirms Streptococcus pneumoniae is...,Pathology confirms Mycoplasma pneumoniae is th...,A 43-year-old woman comes to the physician bec...
3,3,ms-val-3,a 4-year-old is brought into the emeregency ro...,0 A 4-year-old is brought into the emeregency ...,1,4,Laboratory studies confirm that he has a respi...,Laboratory studies confirm that he has parainf...,A 4-year-old is brought into the emeregency ro...
4,4,ms-val-4,a previously healthy 25-year-old man comes to ...,0 A previously healthy 25-year-old man comes t...,1,0,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...,A previously healthy 25-year-old man comes to ...


In [191]:
# Asumiendo new_predictions contiene las predicciones de tu modelo

# Convertir las predicciones de probabilidad en etiquetas binarias
predicted_labels = np.round(new_predictions).astype(int).flatten()

# Añadir las predicciones como una nueva columna en el DataFrame
df['Error'] = predicted_labels


In [192]:
df_final = df

In [193]:
df_final= df_final.drop(['Error Flag'], axis=1)

# Renombrar la columna 'Error Flag Pred' a 'Error Flag'
df_final.rename(columns={'Error': 'Error Flag'}, inplace=True)

# Seleccionar y conservar únicamente las columnas deseadas
df_final = df[['Text ID', 'Error Flag', 'Error Sentence ID', 'Corrected Sentence']]

# Mostrar las primeras filas del DataFrame final para verificar
print(df_final.head())


    Text ID  Error Flag  Error Sentence ID  \
0  ms-val-0           1                  5   
1  ms-val-1           0                 -1   
2  ms-val-2           1                 24   
3  ms-val-3           1                  4   
4  ms-val-4           1                  0   

                                  Corrected Sentence  
0      Culture tests indicate Trichomonas vaginalis.  
1                                                NaN  
2  Pathology confirms Mycoplasma pneumoniae is th...  
3  Laboratory studies confirm that he has parainf...  
4  A previously healthy 25-year-old man comes to ...  


In [194]:
df_final.head()

,Text ID,Error Flag,Error Sentence ID,Corrected Sentence
0,ms-val-0,1,5,Culture tests indicate Trichomonas vaginalis.
1,ms-val-1,0,-1,NaN
2,ms-val-2,1,24,Pathology confirms Mycoplasma pneumoniae is th...
3,ms-val-3,1,4,Laboratory studies confirm that he has parainf...
4,ms-val-4,1,0,A previously healthy 25-year-old man comes to ...


In [195]:
# Crear una nueva columna combinando todas las otras, separadas por espacio
df_final['formatted'] = df_final.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Asegurar un solo espacio entre palabras, incluso si hay múltiples espacios originalmente
df_final['formatted'] = df_final['formatted'].apply(lambda x: ' '.join(x.split()))

# Guardar el DataFrame modificado a un archivo de texto sin comillas y con un solo espacio
with open('prediction.txt', 'w', encoding='utf-8') as file:
    for item in df_final['formatted']:
        file.write("%s\n" % item)

# Ahora, el archivo 'prediction.txt' se guardará en el directorio actual de tu script o notebook.
# Puedes verificar la ubicación actual de tu script con:
import os
print(f"El archivo se ha guardado en: {os.getcwd()}")


El archivo se ha guardado en: c:\Users\USER\Documents\Maestria\CAOBA\Laura


<ipython-input-195-19472edce9b7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['formatted'] = df_final.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
<ipython-input-195-19472edce9b7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['formatted'] = df_final['formatted'].apply(lambda x: ' '.join(x.split()))


In [196]:
df['Error'].unique()

array([0, 1])